# Question 4 

For this question of predictig the category class for the items:
1. We have training Data under Data/train.csv and testing data under Data/test.csv
2. The the variables(columns from the data) are < item_id, sold_price, price, area_name, condition, size, listing_at, item_tag_hash > and the dependent variable is < category_class >

In [137]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import f1_score, make_scorer 

from sklearn.naive_bayes import MultinomialNB  # Multinomial Naive Bayes
from sklearn.linear_model import LogisticRegression # Logistic Regression
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.tree import DecisionTreeClassifier # Decision Tree 

In [89]:
current_path = os.getcwd()
# print(os.getcwd())

In [90]:
master_Train_Data = current_path + '/Data/train.csv'
master_Test_Data = current_path + '/Data/test.csv'

In [97]:
data = pd.read_csv(master_Train_Data, encoding='utf-8')
data.head()

,item_id,category_class,sold_price,price,area_name,condition,size,listing_at,item_tag_hash
0,7966,0.0,1164,1162,fff,Fair,7,2017-02-01 16:11:18.978516,3ca192bd7558780793444f73366c58d60c9d7775
1,1850,0.0,1005,1004,fff,Fair,3,2017-02-02 14:42:51.693295,fbaacb960902382e4f6c96f2d8f225c24eecadb4
2,7582,2.0,944,944,aaa,Fair,5,2017-02-03 01:55:53.406374,785a7925363bf133a7c5413c563f331c5e02cc69
3,6560,0.0,1068,1067,fff,Fair,7,2017-01-30 23:34:02.268603,f8997252c6e5ae3d950b736e1a81160a2e937a7f
4,7368,3.0,1407,1407,ddd,Good,15,2017-02-03 13:38:37.845010,ca59bcd3da4daac52f3fcfdc0ab963f65cf421bc


In [98]:
# converting datetime to individual values
data['listing_at'] = pd.to_datetime(data['listing_at'], format='%Y-%m-%d %H:%M:%S.%f')
data['month'] = data['listing_at'].dt.month
data['year'] = data['listing_at'].dt.year
data['day'] = data['listing_at'].dt.day
data['minute'] = data['listing_at'].dt.minute
data['hour'] = data['listing_at'].dt.hour
data['weekday'] = data['listing_at'].dt.weekday
# converting category_class to ints
data['category_class'] = data.category_class.astype(int)
data

,item_id,category_class,sold_price,price,area_name,condition,size,listing_at,item_tag_hash,month,year,day,minute,hour,weekday
0,7966,0,1164,1162,fff,Fair,7,2017-02-01 16:11:18.978516,3ca192bd7558780793444f73366c58d60c9d7775,2,2017,1,11,16,2
1,1850,0,1005,1004,fff,Fair,3,2017-02-02 14:42:51.693295,fbaacb960902382e4f6c96f2d8f225c24eecadb4,2,2017,2,42,14,3
2,7582,2,944,944,aaa,Fair,5,2017-02-03 01:55:53.406374,785a7925363bf133a7c5413c563f331c5e02cc69,2,2017,3,55,1,4
3,6560,0,1068,1067,fff,Fair,7,2017-01-30 23:34:02.268603,f8997252c6e5ae3d950b736e1a81160a2e937a7f,1,2017,30,34,23,0
4,7368,3,1407,1407,ddd,Good,15,2017-02-03 13:38:37.845010,ca59bcd3da4daac52f3fcfdc0ab963f65cf421bc,2,2017,3,38,13,4
5,9404,2,980,979,ddd,Good,3,2017-01-30 19:33:36.034323,1eb904b6d0c2a41af59e2d1fdd0e10624d94c81a,1,2017,30,33,19,0
6,8367,1,1371,2102,jjj,Good,11,2017-02-01 14:54:04.357796,b7553c6080f116f0115b1f080e03484b98292d8b,2,2017,1,54,14,2
7,7710,2,771,770,hhh,Good,2,2017-02-03 14:16:04.329649,1eb904b6d0c2a41af59e2d1fdd0e10624d94c81a,2,2017,3,16,14,4
8,5402,0,1004,1003,fff,Like New,2,2017-01-31 15:56:31.932041,8422051640d7c9a740564eff35fbdce111c6c557,1,2017,31,56,15,1
9,1909,4,1096,1095,bbb,Good,3,2017-02-01 14:31:52.669176,3b928622f8479780e1593e188d5315edb7d6f786,2,2017,1,31,14,2


In [99]:
# item_tag_hash
lb_item_hash_labels = LabelBinarizer()
lb_results_item_hash_labels = lb_item_hash_labels.fit_transform(data["item_tag_hash"])
item_hash_labels = pd.DataFrame(lb_results_item_hash_labels, columns=lb_item_hash_labels.classes_)
data = pd.merge(data, item_hash_labels, left_index=True, right_index=True)
data.head(n=10)

In [100]:
#condition
lb_condition = LabelBinarizer()
lb_results_condition = lb_condition.fit_transform(data["condition"])
condition_labels = pd.DataFrame(lb_results_condition, columns=lb_condition.classes_)
data = pd.merge(data, condition_labels, left_index=True, right_index=True)
data.head(n=10)

In [101]:
#area_name
lb_area_name = LabelBinarizer()
lb_results_area_name = lb_area_name.fit_transform(data["area_name"])
area_name_labels = pd.DataFrame(lb_results_area_name, columns=lb_area_name.classes_)
data = pd.merge(data, area_name_labels, left_index=True, right_index=True)
data.head(n=10)

,item_id,category_class,sold_price,price,area_name,condition,size,listing_at,item_tag_hash,month,...,aaa,bbb,ccc,ddd,eee,fff,ggg,hhh,jjj,kkk
0,7966,0,1164,1162,fff,Fair,7,2017-02-01 16:11:18.978516,3ca192bd7558780793444f73366c58d60c9d7775,2,...,0,0,0,0,0,1,0,0,0,0
1,1850,0,1005,1004,fff,Fair,3,2017-02-02 14:42:51.693295,fbaacb960902382e4f6c96f2d8f225c24eecadb4,2,...,0,0,0,0,0,1,0,0,0,0
2,7582,2,944,944,aaa,Fair,5,2017-02-03 01:55:53.406374,785a7925363bf133a7c5413c563f331c5e02cc69,2,...,1,0,0,0,0,0,0,0,0,0
3,6560,0,1068,1067,fff,Fair,7,2017-01-30 23:34:02.268603,f8997252c6e5ae3d950b736e1a81160a2e937a7f,1,...,0,0,0,0,0,1,0,0,0,0
4,7368,3,1407,1407,ddd,Good,15,2017-02-03 13:38:37.845010,ca59bcd3da4daac52f3fcfdc0ab963f65cf421bc,2,...,0,0,0,1,0,0,0,0,0,0


In [102]:
# looking at individual datetypes
data.dtypes

item_id                                              int64
category_class                                       int64
sold_price                                           int64
price                                                int64
area_name                                           object
condition                                           object
size                                                 int64
listing_at                                  datetime64[ns]
item_tag_hash                                       object
month                                                int64
year                                                 int64
day                                                  int64
minute                                               int64
hour                                                 int64
weekday                                              int64
00e263ff6806064c016a643c6587b0d607a6a42d             int64
00fd4b4549a1094aae926ef62e9dbd3cdcc2e456             int

In [103]:
# print(data['size'].describe())
# print(data['size'].value_counts())

In [104]:
# data.category_class.value_counts()

In [105]:
# data.area_name.value_counts()

In [106]:
# data.sold_price.value_counts()

In [107]:
# data.condition.value_counts()

In [108]:
# data.price.value_counts()

In [109]:
# data.item_tag_hash.value_counts()

In [110]:
# data

In [111]:
data.corr()

,item_id,category_class,sold_price,price,size,month,year,day,minute,hour,...,aaa,bbb,ccc,ddd,eee,fff,ggg,hhh,jjj,kkk
item_id,1.000000,0.007376,-0.001447,0.018117,-0.016210,0.047450,NaN,-0.051034,0.025038,0.076547,...,-0.058524,-0.023803,0.021431,0.003935,-0.015116,0.027848,0.055431,0.022109,-0.002790,-0.033685
category_class,0.007376,1.000000,0.003725,0.045501,0.156726,-0.073639,NaN,0.074634,-0.070288,-0.025043,...,0.025537,-0.006759,-0.063878,-0.003060,0.045642,-0.046457,0.063891,0.044756,-0.001389,-0.054378
sold_price,-0.001447,0.003725,1.000000,0.776618,0.440042,0.052781,NaN,-0.050606,-0.044959,-0.011497,...,-0.083579,-0.031587,-0.084384,-0.096380,-0.032125,0.039481,0.025001,0.057387,0.092579,0.114904
price,0.018117,0.045501,0.776618,1.000000,0.447971,0.027484,NaN,-0.026450,-0.086014,-0.008191,...,-0.051342,-0.022174,-0.088990,-0.096702,0.001976,-0.002815,0.044040,0.066974,0.080290,0.071914
size,-0.016210,0.156726,0.440042,0.447971,1.000000,0.093477,NaN,-0.087754,-0.092093,-0.018349,...,-0.021859,-0.049289,-0.044128,-0.027482,-0.010392,0.004469,-0.009173,0.059803,0.012192,0.088097
month,0.047450,-0.073639,0.052781,0.027484,0.093477,1.000000,NaN,-0.998299,0.033040,-0.142202,...,0.136585,0.008990,0.025780,-0.085187,0.044110,0.028983,-0.020220,0.030442,-0.034236,-0.122375
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
day,-0.051034,0.074634,-0.050606,-0.026450,-0.087754,-0.998299,NaN,1.000000,-0.030810,0.135228,...,-0.132131,-0.009125,-0.031635,0.087442,-0.043221,-0.032499,0.018933,-0.029842,0.035579,0.123887
minute,0.025038,-0.070288,-0.044959,-0.086014,-0.092093,0.033040,NaN,-0.030810,1.000000,-0.030986,...,-0.012086,0.022494,-0.016151,0.018053,-0.053535,0.011699,-0.007240,-0.043994,0.048457,0.028010
hour,0.076547,-0.025043,-0.011497,-0.008191,-0.018349,-0.142202,NaN,0.135228,-0.030986,1.000000,...,-0.039990,0.004959,-0.011882,-0.003243,0.067886,0.024971,-0.019000,-0.013319,0.016328,-0.023597


In [112]:
# data.columns

In [113]:
independent_variables = data.columns.tolist()
independent_variables.remove('category_class')
independent_variables.remove('listing_at')
independent_variables.remove('year')
independent_variables.remove('item_tag_hash')
independent_variables.remove('condition')
independent_variables.remove('area_name')
dependent_variable = ['category_class']

features = data[independent_variables]
label = data[dependent_variable]

In [114]:
features

,item_id,sold_price,price,size,month,day,minute,hour,weekday,00e263ff6806064c016a643c6587b0d607a6a42d,...,aaa,bbb,ccc,ddd,eee,fff,ggg,hhh,jjj,kkk
0,7966,1164,1162,7,2,1,11,16,2,0,...,0,0,0,0,0,1,0,0,0,0
1,1850,1005,1004,3,2,2,42,14,3,0,...,0,0,0,0,0,1,0,0,0,0
2,7582,944,944,5,2,3,55,1,4,0,...,1,0,0,0,0,0,0,0,0,0
3,6560,1068,1067,7,1,30,34,23,0,0,...,0,0,0,0,0,1,0,0,0,0
4,7368,1407,1407,15,2,3,38,13,4,0,...,0,0,0,1,0,0,0,0,0,0
5,9404,980,979,3,1,30,33,19,0,0,...,0,0,0,1,0,0,0,0,0,0
6,8367,1371,2102,11,2,1,54,14,2,0,...,0,0,0,0,0,0,0,0,1,0
7,7710,771,770,2,2,3,16,14,4,0,...,0,0,0,0,0,0,0,1,0,0
8,5402,1004,1003,2,1,31,56,15,1,0,...,0,0,0,0,0,1,0,0,0,0
9,1909,1096,1095,3,2,1,31,14,2,0,...,0,1,0,0,0,0,0,0,0,0


In [115]:
label

,category_class
0,0
1,0
2,2
3,0
4,3
5,2
6,1
7,2
8,0
9,4


In [116]:
# Splitting the data using Sklearn Model Selection, this should give us better results for cross valication
X_train,X_test,Y_train,Y_test = train_test_split(features, label, test_size=0.33, random_state=1)

## Multinomial Naive Bayes

In [117]:
# Naive Bayes
alpha = [3, 1, 0.3, 0.1, 0.03, 0.01, 0.003]
f1_scorer = make_scorer(f1_score, average='macro')
for i in alpha:
    naive_bayes = MultinomialNB(alpha = i)
    print(cross_val_score(naive_bayes,X_train,Y_train.values.ravel(),cv=3,scoring=f1_scorer))
    

[0.18692297 0.23966327 0.18889308]
[0.16538223 0.24974135 0.186226  ]
[0.18395983 0.27906297 0.19903367]
[0.19021079 0.27906297 0.20594859]
[0.19034701 0.31025061 0.21517242]
[0.19568034 0.31527148 0.21540777]
[0.19568034 0.32482142 0.22366414]


In [118]:
naive_bayes = MultinomialNB(alpha = 0.01)
naive_bayes = naive_bayes.fit(X_train,Y_train.values.ravel())
Y_predicted_naive_bayes = naive_bayes.predict(X_test)

# F1 Score, Precision, Recall
Precision, Recall, F1score, Support = prfs(Y_test, Y_predicted_naive_bayes, average='macro', beta=1.0)
print("Values for Naive Bayes : ","Precision ->",Precision," Recall->",Recall," F1score->",F1score)

Values for Naive Bayes :  Precision -> 0.40796195462317575  Recall-> 0.427088551753345  F1score-> 0.3450244083213879


## Logistic Regression

In [124]:
max_iter = [10000, 100000, 1000000, 10000000]
for i in max_iter:
    linear_regression = LogisticRegression(
                                        penalty='l2', 
                                        tol=0.0001, 
                                        C=1, 
    #                                     class_weight="balanced", 
                                        solver='lbfgs', 
                                        max_iter=i, 
                                        multi_class='multinomial',
                                        n_jobs=2,
                                        verbose=1)
    print(cross_val_score(linear_regression,X_train,Y_train.values.ravel(),cv=3,scoring=f1_scorer))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    6.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    6.1s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    5.5s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


[0.55806869 0.547639   0.58345937]


[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    7.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    7.6s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    7.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


[0.62450397 0.59006315 0.62985865]


[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    7.8s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    7.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    7.6s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


[0.62450397 0.59006315 0.62985865]


[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    7.9s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    7.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


[0.62450397 0.59006315 0.62985865]


[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    7.6s finished


In [127]:
linear_regression = LogisticRegression(
                                        penalty='l2', 
                                        tol=0.0001, 
                                        C=1, 
                                        solver='lbfgs', 
                                        max_iter=100000, 
                                        multi_class='multinomial',
                                        n_jobs=2,
                                        verbose=1)
linear_regression = linear_regression.fit(X_train,Y_train.values.ravel())
Y_predicted_linear_regression = linear_regression.predict(X_test)

# F1 Score, Precision, Recall
Precision, Recall, F1score, Support = prfs(Y_test, Y_predicted_linear_regression, average='macro', beta=1.0)
print("Values for Logistic Regression : ","Precision ->",Precision," Recall->",Recall," F1score->",F1score)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Values for Logistic Regression :  Precision -> 0.7183712121212121  Recall-> 0.6446269427080347  F1score-> 0.6663927628386265


[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    9.8s finished


## Random Forset

In [140]:
random_forest = RandomForestClassifier(n_estimators=1000, 
                                       criterion='gini',
                                       n_jobs=2,
#                                        class_weight="balanced",
                                       verbose=1)
print(cross_val_score(random_forest, X_train, Y_train.values.ravel(), cv=3, scoring=f1_scorer))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | ela

[0.61664891 0.6149503  0.57120203]


In [143]:
random_forest = RandomForestClassifier(n_estimators=1000, 
                                       criterion='gini',
                                       n_jobs=2,
#                                        class_weight="balanced",
                                       verbose=1)
random_forest = random_forest.fit(X_train,Y_train.values.ravel())
Y_predicted_random_forest = random_forest.predict(X_test)

# F1 Score, Precision, Recall
Precision, Recall, F1score, Support = prfs(Y_test, Y_predicted_random_forest, average='macro', beta=1.0)
print("Values for random_forest : ","Precision ->",Precision," Recall->",Recall," F1score->",F1score)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s


Values for random_forest :  Precision -> 0.6767219067995767  Recall-> 0.591782284355548  F1score-> 0.612128218602207


[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


## Decision Tree

In [146]:
decision_tree = DecisionTreeClassifier(class_weight="balanced", 
                                    presort=True)
print(cross_val_score(decision_tree, X_train, Y_train.values.ravel(), cv=3, scoring=f1_scorer))

[0.45461245 0.45166978 0.43912389]


In [148]:
decision_tree = decision_tree.fit(X_train,Y_train.values.ravel())
Y_predicted_decision_tree = decision_tree.predict(X_test)

# F1 Score, Precision, Recall
Precision, Recall, F1score, Support = prfs(Y_test, Y_predicted_decision_tree, average='macro', beta=1.0)
print("Values for decision_tree : ","Precision ->",Precision," Recall->",Recall," F1score->",F1score)

Values for decision_tree :  Precision -> 0.5463882388398397  Recall-> 0.5464328285123147  F1score-> 0.5448302011716384
